In [1]:
import os
import numpy as np # to process the data
import random
import math
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
class FlowerDataset:
    def __init__(self, directory):
        self.directory = directory
        self.images = []
        self.labels = []

    def load_data(self):
        for i, img in enumerate(os.listdir(self.directory)):
            if i >= 10:  # only load the first 10 images
                break
            try:
                img_array = Image.open(os.path.join(self.directory, img))  # read the image
                gray_array = img_array.convert('L')  # ensure image is grayscale
                resized_array = gray_array.resize((150, 150))  # resize the image
                self.images.append(np.array(resized_array))
                label = img.split('_')[0]  # extract label from filename
                self.labels.append(label)
                # print(f"Loaded image {img} with label {label}")
            except Exception as e:
                print(f"Error loading image {img}: {e}")

    def preprocess_data(self):
        self.images = np.array(self.images).reshape(-1, 150, 150, 1)  # update the reshape dimensions for grayscale
        self.images = self.images / 255.0  # normalize pixel values
        self.labels = np.array(self.labels)
        print(f"Loaded {len(self.images)} images")


# Load and preprocess the data
dataset = FlowerDataset('flowers')
dataset.load_data()
dataset.preprocess_data()

print(f"Shape of the preprocessed images: {dataset.images.shape}")
print(f"Dimensions of the first preprocessed image: {dataset.images[0].shape}") # dimensions of first image

# Get the images and labels
images = dataset.images
labels = dataset.labels

# Convert labels to numerical values
unique_labels = list(set(labels))
labels_encoded = [unique_labels.index(l) for l in labels]

# One-hot encoding function
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

# One-hot encode the labels
labels_encoded_one_hot = one_hot_encode(labels_encoded)

# Split the data into training, validation, and test sets
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(images, labels_encoded_one_hot, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

print(f"Train set size: {len(x_train)}, Validation set size: {len(x_val)}, Test set size: {len(x_test)}")

# Convert one-hot encoded labels back to class indices
y_train_indices = np.argmax(y_train, axis=1)

# Print the unique classes in the training set
unique_classes = set(y_train_indices)
print(f"Unique classes in the training set: {unique_classes}")

Loaded 10 images
Shape of the preprocessed images: (10, 150, 150, 1)
Dimensions of the first preprocessed image: (150, 150, 1)
Train set size: 7, Validation set size: 1, Test set size: 2
Unique classes in the training set: {0, 1, 2, 3}


In [7]:
def relu(x):
    if isinstance(x, list):
        return [max(0, xi) for xi in x]
    else:
        return max(0, x)

def relu_derivative(x):
    if isinstance(x, list):
        return [1 if xi > 0 else 0 for xi in x]
    else:
        return 1 if x > 0 else 0

def mean_squared_error(y_true, y_pred):
    return sum((yt - yp) ** 2 for yt, yp in zip(y_true, y_pred)) / len(y_true)

def gradient_mse(y_true, y_pred):
    return [2 * (yp - yt) for yt, yp in zip(y_true, y_pred)]

def softmax(x):
    # Check if x is a list of lists
    if isinstance(x[0], list):
        # Apply softmax to each list of logits in x
        return [softmax(xi) for xi in x]
    else:
        # Apply softmax to a single list of logits
        max_x = max(x)
        exps = [math.exp(i - max_x) for i in x]
        sum_of_exps = sum(exps)
        return [j / sum_of_exps for j in exps]

def cross_entropy(predictions, targets):
    # Assuming predictions is a list of probabilities and targets is a list of one-hot encoded classes
    N = len(predictions)
    ce = 0
    for p, t in zip(predictions, targets):
        ce -= t * math.log(p) if p > 0 else 0  # Adding check to prevent math domain error
    ce /= N
    return ce
     
# Layer classes
class Layer:
    def forward(self, input):
        raise NotImplementedError

    def backward(self, input, gradient):
        raise NotImplementedError

class Conv2D(Layer):
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = [[random.random() for _ in range(filter_size * filter_size)] for _ in range(num_filters)]

    def forward(self, input):
        height, width = len(input), len(input[0])
        output_height = height - self.filter_size + 1
        output_width = width - self.filter_size + 1
        output = [[0 for _ in range(output_width)] for _ in range(output_height)]

        for i in range(output_height):
            for j in range(output_width):
                for f in range(self.num_filters):
                    output[i][j] += sum(
                        input[i + m][j + n] * self.filters[f][m * self.filter_size + n]
                        for m in range(self.filter_size)
                        for n in range(self.filter_size)
                    )
        return output

    def backward(self, input, gradient):
        d_filters = [[[0 for _ in range(self.filter_size * self.filter_size)] for _ in range(self.num_filters)]]
        for i in range(len(input) - self.filter_size + 1):
            for j in range(len(input[0]) - self.filter_size + 1):
                for f in range(self.num_filters):
                    for m in range(self.filter_size):
                        for n in range(self.filter_size):
                            d_filters[f][m * self.filter_size + n] += input[i + m][j + n] * gradient[i][j]
        return d_filters

class MaxPooling2D(Layer):
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, input):
        height, width = len(input), len(input[0])
        new_height = height // self.pool_size
        new_width = width // self.pool_size
        output = [[0 for _ in range(new_width)] for _ in range(new_height)]

        for i in range(new_height):
            for j in range(new_width):
                output[i][j] = max(
                    input[i * self.pool_size + m][j * self.pool_size + n]
                    for m in range(self.pool_size)
                    for n in range(self.pool_size)
                )
        return output

    def backward(self, input, gradient):
        d_input = [[0 for _ in range(len(input[0]))] for _ in range(len(input))]
        for i in range(0, len(input), self.pool_size):
            for j in range(0, len(input[0]), self.pool_size):
                window = [input[i + m][j + n] for m in range(self.pool_size) for n in range(self.pool_size)]
                max_value = max(window)
                for m in range(self.pool_size):
                    for n in range(self.pool_size):
                        if input[i + m][j + n] == max_value:
                            d_input[i + m][j + n] = gradient[i // self.pool_size][j // self.pool_size]
        return d_input

class Flatten(Layer):
    def forward(self, input):
        self.input_shape = (len(input), len(input[0]))  # save this for backward pass
        return [item for sublist in input for item in sublist]  # flatten the list

    def backward(self, input, gradient):
        return [gradient[i:i+self.input_shape[1]] for i in range(0, len(gradient), self.input_shape[1])]

class Dense(Layer):
    def __init__(self, num_inputs, num_outputs):
        # Initialize weights and biases
        self.weights = [[random.random() for _ in range(num_inputs)] for _ in range(num_outputs)]
        self.biases = [random.random() for _ in range(num_outputs)]

    def forward(self, input):
        # Compute the weighted sum of inputs plus biases
        return [[sum(i * w + b for i, w in zip(input_sample, weights)) + b for weights, b in zip(self.weights, self.biases)] for input_sample in input]

    def backward(self, input, gradient):
        # Compute gradients with respect to weights and biases
        d_weights = [[i * g for i in input] for g in gradient]
        d_biases = gradient
        d_input = [0 for _ in range(len(input))]
        for i, g in enumerate(gradient):
            for j, w in enumerate(self.weights[i]):
                d_input[j] += g * w
        return d_input, d_weights, d_biases

class Dropout(Layer):
    def __init__(self, rate):
        self.rate = rate

    def forward(self, input):
        self.input_shape = (len(input), len(input[0]))  # save this for backward pass
        self.mask = [[random.random() > self.rate for _ in range(self.input_shape[1])] for _ in range(self.input_shape[0])]
        return [[input[i][j] * self.mask[i][j] for j in range(self.input_shape[1])] for i in range(self.input_shape[0])]

    def backward(self, input, gradient):
        return [[gradient[i][j] * self.mask[i][j] for j in range(self.input_shape[1])] for i in range(self.input_shape[0])]

# Activation functions as layers
class ReLU(Layer):
    def forward(self, input):
        # Apply ReLU activation element-wise
        return [relu(x) for x in input]

    def backward(self, input, gradient):
        # Apply the derivative of ReLU to the gradient
        return [g * relu_derivative(x) for x, g in zip(input, gradient)]

# Loss function as a class
class SoftmaxCrossEntropyLoss:
    def forward(self, logits, labels):
        self.predictions = softmax(logits)
        return cross_entropy(self.predictions, labels)

    def backward(self, logits, labels):
        return [p - l for p, l in zip(self.predictions, labels)]

In [8]:
class CustomCNNModel:
    def __init__(self):
        self.layers = [
            Conv2D(32, 3),
            ReLU(),
            MaxPooling2D(2),
            Conv2D(64, 3),
            ReLU(),
            MaxPooling2D(2),
            Conv2D(128, 3),
            ReLU(),
            MaxPooling2D(2),
            Flatten(),
            Dense(128, 128),
            ReLU(),
            Dropout(0.1),
            Dense(128, 3)
        ]
        self.loss = SoftmaxCrossEntropyLoss()

    def forward(self, input, labels):
        self.last_input_shape = input.shape
        for layer in self.layers:
            input = layer.forward(input)
        return self.loss.forward(input, labels)

    def backward(self, logits, labels):
        gradient = self.loss.backward(logits, labels)
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)

    def train(self, x_train, y_train, x_val, y_val, epochs, learning_rate, threshold, early_stopping):
        history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
        best_val_loss = float('inf')
        no_improvement = 0

        for epoch in range(epochs):
            # Training phase
            train_loss = 0
            train_correct = 0
            for x, y in zip(x_train, y_train):
                logits = self.forward(x, y)
                train_loss += self.loss.forward(logits, y)
                train_correct += np.argmax(logits) == np.argmax(y)
                gradient = self.backward(logits, y)
                for layer in self.layers:
                    if isinstance(layer, (Conv2D, Dense)):
                        layer.weights -= learning_rate * gradient

            train_loss /= len(x_train)
            train_acc = train_correct / len(x_train)
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)

            # Validation phase
            val_loss = 0
            val_correct = 0
            for x, y in zip(x_val, y_val):
                logits = self.forward(x, y)
                val_loss += self.loss.forward(logits, y)
                val_correct += np.argmax(logits) == np.argmax(y)

            val_loss /= len(x_val)
            val_acc = val_correct / len(x_val)
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)

            print(f"Epoch {epoch+1}/{epochs}")
            print(f"Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}")
            print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}")

            # Early stopping
            if val_loss < best_val_loss - threshold:
                best_val_loss = val_loss
                no_improvement = 0
            else:
                no_improvement += 1
            if no_improvement >= early_stopping:
                print(f"Stopping training after {epoch+1} epochs due to no improvement in validation loss.")
                break

        return history



# Define your training parameters
epochs = 10  # Number of epochs to train for
learning_rate = 1  # Learning rate
threshold = 0.01  # Threshold for early stopping
early_stopping = 10  # Number of rounds without improvement before early stopping

# Create an instance of the model
model = CustomCNNModel()

# Train the model
history = model.train(x_train, y_train, x_val, y_val, epochs, learning_rate, threshold, early_stopping)

TypeError: '>' not supported between instances of 'list' and 'int'